In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Head_and_Neck_Cancer"
cohort = "GSE156915"

# Input paths
in_trait_dir = "../../input/GEO/Head_and_Neck_Cancer"
in_cohort_dir = "../../input/GEO/Head_and_Neck_Cancer/GSE156915"

# Output paths
out_data_file = "../../output/preprocess/Head_and_Neck_Cancer/GSE156915.csv"
out_gene_data_file = "../../output/preprocess/Head_and_Neck_Cancer/gene_data/GSE156915.csv"
out_clinical_data_file = "../../output/preprocess/Head_and_Neck_Cancer/clinical_data/GSE156915.csv"
json_path = "../../output/preprocess/Head_and_Neck_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"In-depth clinical and biological exploration of DNA Damage Immune Response (DDIR) as a biomarker for oxaliplatin use in colorectal cancer"
!Series_summary	"Purpose: The DNA Damage Immune Response (DDIR) assay was developed in breast cancer (BC) based on biology associated with deficiencies in homologous recombination and Fanconi Anemia (HR/FA) pathways. A positive DDIR call identifies patients likely to respond to platinum-based chemotherapies in breast and oesophageal cancers. In colorectal cancer (CRC) there is currently no biomarker to predict response to oxaliplatin. We tested the ability of the DDIR assay to predict response to oxaliplatin-based chemotherapy in CRC and characterised the biology in DDIR-positive CRC."
!Series_summary	"Methods: Samples and clinical data were assessed according to DDIR status from patients who received either 5FU or FOLFOX within the FOCUS trial (n=361, stage 4), or neo-adjuvant FOLFOX in the FOxTROT trial (n=97

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import numpy as np
import os
import re
from typing import Dict, Any, Optional, Callable

# 1. Gene Expression Data Availability
# Based on the background information, this dataset appears to be gene expression data
# The study explores DNA Damage Immune Response and has transcriptome data
is_gene_available = True

# 2.1 Data Availability
# From the sample characteristics dictionary:

# For trait (Head and Neck Cancer):
# There's no direct mention of head and neck cancer in the sample characteristics
# This dataset appears to be for colorectal cancer, not head and neck cancer
trait_row = None

# For age:
# No age information is present in the sample characteristics
age_row = None

# For gender:
# No gender information is present in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion
# Since trait data is not available for head and neck cancer, we'll create a simple function
def convert_trait(value):
    if value is None or pd.isna(value):
        return None
    # Extract value after colon if present
    if ':' in str(value):
        value = value.split(':', 1)[1].strip()
    # This would be a placeholder as the dataset doesn't contain head and neck cancer information
    return None

# Age conversion function (not used but defined for completeness)
def convert_age(value):
    if value is None or pd.isna(value):
        return None
    # Extract value after colon if present
    if ':' in str(value):
        value = value.split(':', 1)[1].strip()
    try:
        return float(value)  # Continuous variable
    except:
        return None

# Gender conversion function (not used but defined for completeness)
def convert_gender(value):
    if value is None or pd.isna(value):
        return None
    # Extract value after colon if present
    if ':' in str(value):
        value = value.split(':', 1)[1].strip()
    # Convert to binary: female=0, male=1
    value = value.lower()
    if 'female' in value or 'f' == value:
        return 0
    elif 'male' in value or 'm' == value:
        return 1
    return None

# 3. Save Metadata
# Initial filtering to determine if dataset is usable
# Since trait_row is None, the trait data is not available for head and neck cancer
validate_and_save_cohort_info(
    is_final=False, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available,
    is_trait_available=(trait_row is not None)
)

# 4. Clinical Feature Extraction
# Since trait_row is None, we skip this substep


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Head_and_Neck_Cancer/GSE156915/GSE156915_series_matrix.txt.gz


Gene data shape: (27054, 361)
First 20 gene/probe identifiers:
Index(['1060P11.3 /// KIR3DP1', 'A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1',
       'A2ML1', 'A2MP1', 'A3GALT2', 'A4GALT', 'A4GNT', 'AA06', 'AAAS', 'AACS',
       'AACSP1', 'AADAC', 'AADACL2', 'AADACL3', 'AADACL4', 'AADACP1'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Analyze gene identifiers
# These appear to be standard human gene symbols (like A1BG, AAAS, AACS, etc.)
# The format follows official HGNC gene symbol nomenclature
# There are some composite identifiers (e.g. "1060P11.3 /// KIR3DP1") that contain
# multiple gene symbols separated by "///" which is a common format in microarray data
# indicating cross-hybridization, but the identifiers themselves are gene symbols

requires_gene_mapping = False


### Step 5: Data Normalization and Linking

In [6]:
# 1. Normalize gene symbols in the gene expression data
# Use normalize_gene_symbols_in_index to standardize gene symbols
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape before normalization: {gene_data.shape}")
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Save the normalized gene data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene expression data saved to {out_gene_data_file}")

# Since we determined in Step 2 that this dataset doesn't contain our target trait (Head and Neck Cancer),
# we should validate this and finish the process without attempting to link data.

# 5. Conduct final quality validation
note = "Dataset contains gene expression data related to colorectal cancer and DNA Damage Immune Response (DDIR), not Head and Neck Cancer."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=False,  # We confirmed trait_row is None in Step 2
    is_biased=True,  # Set to True since we can't use data without our target trait
    df=pd.DataFrame(),  # Empty DataFrame since we're not processing linked data
    note=note
)

print("Dataset is not usable for Head and Neck Cancer analysis as it contains data for a different trait (colorectal cancer).")

Gene data shape before normalization: (27054, 361)
Gene data shape after normalization: (22171, 361)


Normalized gene expression data saved to ../../output/preprocess/Head_and_Neck_Cancer/gene_data/GSE156915.csv
Abnormality detected in the cohort: GSE156915. Preprocessing failed.
Dataset is not usable for Head and Neck Cancer analysis as it contains data for a different trait (colorectal cancer).
